Source for scraping: zillow

In [48]:
import pandas as pd
import requests
import re
import json
from bs4 import BeautifulSoup

In [49]:
url = 'https://www.zillow.com/ca/'
header = {"User-Agent": "Chrome 109.0.0.0","referer":'https://www.zillow.com/ca/' }

In [50]:
r = requests.get(url=url,headers=header)

In [51]:
r

<Response [200]>

In [52]:
soup = BeautifulSoup(r.text,'html.parser')
script_tag = soup.find('script', {'id': '__NEXT_DATA__'})

In [53]:
url2 = url+ soup.find('a',class_= "StyledButton-c11n-8-109-3__sc-iv7357-0 jRehVC PaginationButton-c11n-8-109-3__sc-1i6hxyy-0 dmzPQJ").get('href')

In [54]:
print(url2)

https://www.zillow.com/ca//ca/2_p/


In [55]:
for i in range(1,11):
    a = 'https://www.zillow.com/ca//ca/'+str(i)+'_p/'
    print(a)



https://www.zillow.com/ca//ca/1_p/
https://www.zillow.com/ca//ca/2_p/
https://www.zillow.com/ca//ca/3_p/
https://www.zillow.com/ca//ca/4_p/
https://www.zillow.com/ca//ca/5_p/
https://www.zillow.com/ca//ca/6_p/
https://www.zillow.com/ca//ca/7_p/
https://www.zillow.com/ca//ca/8_p/
https://www.zillow.com/ca//ca/9_p/
https://www.zillow.com/ca//ca/10_p/


In [56]:
url1 = 'https://www.zillow.com/ca//ca/1_p/'
url2 = 'https://www.zillow.com/ca//ca/2_p/'
url3 = 'https://www.zillow.com/ca//ca/3_p/'
url4 = 'https://www.zillow.com/ca//ca/4_p/'
url5 = 'https://www.zillow.com/ca//ca/5_p/'
url6 = 'https://www.zillow.com/ca//ca/6_p/'
url7 = 'https://www.zillow.com/ca//ca/7_p/'
url8 = 'https://www.zillow.com/ca//ca/8_p/'
url9 = 'https://www.zillow.com/ca//ca/9_p/'
url10 = 'https://www.zillow.com/ca//ca/10_p/'

In [57]:
urls = [url1,url2, url3, url4, url5, url6, url7, url8, url9, url10]


In [58]:
with requests.Session() as s:
    r1 = s.get(url1,headers=header)
    r2 = s.get(url2,headers=header)
    r3 = s.get(url3,headers=header)
    r4 = s.get(url4,headers=header)
    r5 = s.get(url5,headers=header)
    r6 = s.get(url6,headers=header)
    r7 = s.get(url7,headers=header)
    r8 = s.get(url8,headers=header)
    r9 = s.get(url9,headers=header)
    r10 = s.get(url10,headers=header)


In [59]:
print(r1.text)


<!DOCTYPE html><html lang="en"><head><script id="scripts.pfs.appInfo">
        if (typeof Object.assign === 'function') {
            window.appInfo = Object.assign(
                typeof window.appInfo === 'object' ? window.appInfo : {},
                {"@zillow/page-frame-content":"c6287115"}
            );
        }
    </script><script id="scripts.client-profiler.config">
        window.CLIENT_PROFILER_CONFIG = window.CLIENT_PROFILER_CONFIG || {
            staticDimensions: {
                ABDecisionToken: "tUk5BT_1759262157",
            },
        };
    </script><script id="scripts.ua">(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
        (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
        m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
        })(window,document,'script','https://www.google-analytics.com/analytics.js','ga');</script><script id="scripts.ua.init">window.ga=windo

In [60]:
m = re.search(r'!--(\{"queryState".*?\})-->', r1.text,re.DOTALL)
if m:
    data1 = json.loads(m.group(1))
else:
    print("No match for r1")


No match for r1


In [61]:
responses = [r1, r2, r3, r4, r5, r6, r7, r8, r9, r10]
datas = []
for i, r in enumerate(responses, 1):
    m = re.search(r'!--("queryState".*?)-->', r.text, re.DOTALL)
    if m:
        datas.append(json.loads(m.group(1)))
    else:
        print(f"No match for r{i}")


No match for r1
No match for r2
No match for r3
No match for r4
No match for r5
No match for r6
No match for r7
No match for r8
No match for r9
No match for r10


In [62]:
all_listing_data = []
for response in responses:
    if response.status_code == 200:
        
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'id': '__NEXT_DATA__'})
        
        if script_tag:
            
            json_data = json.loads(script_tag.string)
            list_results = json_data.get('props', {}).get('pageProps', {}).get('searchPageState', {}).get('cat1', {}).get('searchResults', {}).get('listResults', [])
            
            if list_results:
                all_listing_data.extend(list_results)
            else:
                print("Could not find 'listResults' in the JSON data for one of the pages.")
        else:
            print("Could not find the '__NEXT_DATA__' script tag on one of the pages.")
    else:
        print(f"Request failed with status code: {response.status_code}")

if all_listing_data:
    df = pd.DataFrame(all_listing_data)
    print(f"Successfully scraped {len(df)} listings.")
   

Successfully scraped 410 listings.


In [63]:
print("\nFirst 5 rows of the DataFrame:")
pd.set_option('display.max_columns', None) # Ensure all columns are displayed
print(df.head())


First 5 rows of the DataFrame:
        zpid               palsId         id rawHomeStatusCd  \
0   16947731      11001_250039916   16947731         ForSale   
1   19698482  18932001_ML82022823   19698482         ForSale   
2   20032553      3009_SR25182617   20032553         ForSale   
3   99586844      3009_SW25215603   99586844         ForSale   
4  448287886    31840001_41112970  448287886         ForSale   

  marketingStatusSimplifiedCd  \
0           For Sale by Agent   
1           For Sale by Agent   
2           For Sale by Agent   
3           For Sale by Agent   
4           For Sale by Agent   

                                              imgSrc hasImage  \
0  https://photos.zillowstatic.com/fp/46467b27828...     True   
1  https://photos.zillowstatic.com/fp/3a1de0eec11...     True   
2  https://photos.zillowstatic.com/fp/73750187c87...     True   
3  https://photos.zillowstatic.com/fp/207fa1270e2...     True   
4  https://photos.zillowstatic.com/fp/c7212d2ddbf...     Tr

In [64]:
df.columns

Index(['zpid', 'palsId', 'id', 'rawHomeStatusCd',
       'marketingStatusSimplifiedCd', 'imgSrc', 'hasImage', 'detailUrl',
       'statusType', 'statusText', 'countryCurrency', 'price',
       'unformattedPrice', 'address', 'addressStreet', 'addressCity',
       'addressState', 'addressZipcode', 'isUndisclosedAddress', 'beds',
       'baths', 'area', 'latLong', 'isZillowOwned', 'flexFieldText',
       'contentType', 'hdpData', 'isSaved', 'isUserClaimingOwner',
       'isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate',
       'shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec',
       'hasAdditionalAttributions', 'isFeaturedListing', 'isShowcaseListing',
       'list', 'relaxed', 'brokerName', 'carouselPhotosComposable',
       'isPaidBuilderNewConstruction', 'hasOpenHouse', 'openHouseStartDate',
       'openHouseEndDate', 'openHouseDescription', 'availabilityDate',
       'info6String', 'streetViewURL', 'streetViewMetadataURL',
       'providerListingId', 'info3Strin

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 56 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   zpid                          410 non-null    object 
 1   palsId                        410 non-null    object 
 2   id                            410 non-null    object 
 3   rawHomeStatusCd               410 non-null    object 
 4   marketingStatusSimplifiedCd   410 non-null    object 
 5   imgSrc                        410 non-null    object 
 6   hasImage                      405 non-null    object 
 7   detailUrl                     410 non-null    object 
 8   statusType                    410 non-null    object 
 9   statusText                    410 non-null    object 
 10  countryCurrency               410 non-null    object 
 11  price                         410 non-null    object 
 12  unformattedPrice              410 non-null    int64  
 13  addre

In [66]:
required = df[['id','price','addressStreet','addressCity','addressZipcode','beds','baths','area','latLong','zestimate']]

In [67]:
required.isnull().sum()

id                 0
price              0
addressStreet      0
addressCity        0
addressZipcode     0
beds               0
baths              1
area               0
latLong            0
zestimate         73
dtype: int64

In [68]:
required['latitude'] = required['latLong'].apply(lambda x: x.get('latitude') if isinstance(x, dict) else None)
required['longitude'] = required['latLong'].apply(lambda x: x.get('longitude') if isinstance(x, dict) else None)

C:\Users\abhin\AppData\Local\Temp\ipykernel_1408\316558378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required['latitude'] = required['latLong'].apply(lambda x: x.get('latitude') if isinstance(x, dict) else None)
C:\Users\abhin\AppData\Local\Temp\ipykernel_1408\316558378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required['longitude'] = required['latLong'].apply(lambda x: x.get('longitude') if isinstance(x, dict) else None)


In [69]:
required.loc[:,'price'] = pd.to_numeric(required['price'].astype('str').str.replace('$','').str.replace(',',''),errors = 'coerce')
required.loc[:,'price'] = required.loc[:,'price'].fillna(0)

C:\Users\abhin\AppData\Local\Temp\ipykernel_1408\24630893.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  required.loc[:,'price'] = required.loc[:,'price'].fillna(0)


In [70]:
required = required.drop('latLong',axis=1)

In [71]:
required.head()

,id,price,addressStreet,addressCity,addressZipcode,beds,baths,area,zestimate,latitude,longitude
0,16947731,1499000.0,5232 Cromwell Ct,San Diego,92116,3,2.0,1198,1470500.0,32.774590,-117.118675
1,19698482,998888.0,1398 Boysea Dr,San Jose,95118,3,2.0,1180,NaN,37.260230,-121.889420
2,20032553,1399000.0,3969 Alta Mesa Dr,Studio City,91604,3,2.0,1200,1393700.0,34.133926,-118.408660
3,99586844,939999.0,1671 Joshua Tree Ln,Fallbrook,92028,3,3.0,2055,937200.0,33.360720,-117.223660
4,448287886,699000.0,3927 Aqua Vista St,Oakland,94601,4,3.0,2369,696800.0,37.782560,-122.208820


In [72]:
required.to_csv(r'data/processed/scraped_california.csv')